In [1]:
import os
import shutil
import warnings

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import torchaudio

warnings.filterwarnings(action="ignore")
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import json

import geopandas
import librosa
import librosa.display
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns
import soundfile
import tensorflow as tf
import tensorflow_addons as tfa
import torch

%matplotlib inline
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from torchaudio.transforms import MelSpectrogram
from tqdm import tqdm

from src.gen_short import MEL_Generator_Short, Mel_Provider
from src.prepare_dataset import make_dict_birds, prepare_dataset, choose_ids

In [2]:
SEED = 42
IMG_SIZE = 260
SAMPLE_RATE = 32000
N_FFT = 2048
SIGNAL_LENGTH = 5
FREQ_MIN = 500
FREQ_MAX = 15000
WIN_LENGHT = 1024
BATCH_SIZE = 128

In [3]:
rand_cut = pd.read_csv("/app/_data/random_cut_edited.csv")
train_sound = pd.read_csv("/app/_data/train_soundscapes_prepared.csv")
train_sound = prepare_dataset(train_sound)
rand_cut = prepare_dataset(rand_cut)
rand_cut.shape

(686847, 21)

In [4]:
_, list_filenames = choose_ids(500)
len(list_filenames)

21416

In [5]:
rand_cut = rand_cut.query('filename in @list_filenames')
rand_cut.shape

(221458, 21)

In [6]:
all_audio = pd.concat([rand_cut, train_sound], axis=0, ignore_index=True)
all_audio = all_audio[all_audio["rating"] >= 3]
all_audio.shape
all_audio = all_audio.reset_index(drop=True)

(191256, 21)

In [7]:
dict_birds, df = make_dict_birds(all_audio)

In [8]:
df.sample(3)

,filename,primary_label,secondary_labels,label_id,secondary_labels_id,start_sec,end_sec,row_id,duration,rating,...,year,file_path,frames,sin_month,cos_month,sin_longitude,cos_longitude,latitude,norm_latitude,date
117701,XC358899.ogg,oaktit,NaN,226,NaN,26.62,31.62,XC358899_31_62,36.901000,4.0,...,2017,/app/_data/train_short_audio/oaktit/XC358899.ogg,1180832.0,8.660254e-01,0.500000,-0.897477,-0.441061,34.0148,0.688971,2017-02-16
132186,XC322509.ogg,reevir1,rebwoo acafly,263,261 0,7.60,12.60,XC322509_12_6,24.177437,4.5,...,2016,/app/_data/train_short_audio/reevir1/XC322509.ogg,773678.0,5.000000e-01,-0.866025,-0.968333,0.249663,39.1742,0.717634,2016-05-15
170700,XC436105.ogg,wesant1,NaN,349,NaN,4.18,9.18,XC436105_9_19,69.229000,4.0,...,2017,/app/_data/train_short_audio/wesant1/XC436105.ogg,2215328.0,1.224647e-16,-1.000000,-0.969591,0.244730,6.4764,0.535980,2017-06-27


In [9]:
file_json = open("/app/_data/models/Eff02_2/dict_birds_eff2_02.json", "w")
json.dump(dict_birds, file_json)
file_json.close()

In [10]:
train, valid = train_test_split(df, train_size=0.85, random_state=SEED)
train = train.reset_index(drop=True)
valid = valid.reset_index(drop=True)

In [11]:
mel_pr = Mel_Provider(
    n_fft=N_FFT,
    win_length=WIN_LENGHT,
    n_mels=IMG_SIZE,
    sample_rate=SAMPLE_RATE,
    mel_image_size=IMG_SIZE,
    min_frequency=FREQ_MIN,
    max_frequency=FREQ_MAX,
    signal_lenght=SIGNAL_LENGTH,
)

In [12]:
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor="val_f1_score",
        patience=12,
        restore_best_weights=True,
        verbose=1,
        mode="max",
    ),
    keras.callbacks.ModelCheckpoint(
        "/app/_data/models/Eff02_2/eff02_2.h5",
        monitor="val_f1_score",
        verbose=1,
        save_best_only=True,
        save_weights_only=False,
        mode="max",
        save_freq="epoch",
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor="val_f1_score",
        factor=0.9,
        patience=5,
        verbose=1,
        mode="max",
        min_delta=1e-4,
        min_lr=0.00000001,
    ),
    keras.callbacks.TensorBoard(log_dir="/app/.tensorboard/eff0_2/", histogram_freq=0),
    keras.callbacks.experimental.BackupAndRestore("/app/_data/models/Eff02_2/backup/"),
    keras.callbacks.TerminateOnNaN(),
]

In [13]:
def get_model():
    inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    base_model = keras.applications.EfficientNetB2(
        weights="imagenet", include_top=False
    )
    x = base_model(inputs)
    x = keras.layers.GlobalAveragePooling2D(name="avg_pool")(x)
    x = keras.layers.Flatten(name="flatten")(x)
    outputs = keras.layers.Dense(NUM_CLASSES, activation="sigmoid")(x)
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        loss="binary_crossentropy",
        optimizer=Adam(lr=0.005),
        metrics=[
            "acc",
            keras.metrics.Recall(),
            keras.metrics.Precision(),
            tfa.metrics.F1Score(num_classes=NUM_CLASSES, average="micro"),
        ],
    )
    return model

In [14]:
policy = keras.mixed_precision.experimental.Policy("mixed_float16")
keras.mixed_precision.experimental.set_policy(policy)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3090, compute capability 8.6


In [15]:
# for i in [
#     "/app/_data/npy/waves_npy/",
#     "/app/_data/npy/short_mels/",
# ]:
#     shutil.rmtree(i)

In [16]:
NUM_CLASSES = len(dict_birds)

In [17]:
gen_train = MEL_Generator_Short(
    df=train,
    n_mels=IMG_SIZE,
    seed=SEED,
    sample_rate=SAMPLE_RATE,
    mel_image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    n_classes=NUM_CLASSES,
    signal_lenght=SIGNAL_LENGTH,
    mel_provider=mel_pr,
    wave_dir="/app/_data/npy/waves_npy/",
    short_mel_dir="/app/_data/npy/short_mels/",
    norm_mel=True,
    convert_to_rgb=True,
    shuffle=True,
    return_primary_labels=False,
    return_secondary_labels=False,
    return_concat_labels=True,
    augment=False
)

gen_valid = MEL_Generator_Short(
    df=valid,
    n_mels=IMG_SIZE,
    seed=SEED,
    sample_rate=SAMPLE_RATE,
    mel_image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    n_classes=NUM_CLASSES,
    signal_lenght=SIGNAL_LENGTH,
    mel_provider=mel_pr,
    wave_dir="/app/_data/npy/waves_npy/",
    short_mel_dir="/app/_data/npy/short_mels/",
    norm_mel=True,
    convert_to_rgb=True,
    shuffle=False,
    return_primary_labels=False,
    return_secondary_labels=False,
    return_concat_labels=True,
    augment=False
)

In [18]:
model = keras.models.load_model('/app/_data/models/Eff02_2/eff02_2_1.h5')

In [ ]:
history = model.fit(
    gen_train,
    validation_data=gen_valid,
    epochs=200,
    steps_per_epoch=train.shape[0] // BATCH_SIZE,
    validation_steps=valid.shape[0] // BATCH_SIZE,
    verbose=1,
    workers=30,
    callbacks=callbacks,
)

Epoch 1/200
   1/1270 [..............................] - ETA: 0s - loss: 0.0016 - acc: 1.0000 - recall: 0.5139 - precision: 1.0000 - f1_score: 0.8740WARNING:tensorflow:From /usr/local/lib/python3.8/dist-packages/tensorflow/python/ops/summary_ops_v2.py:1277: stop (from tensorflow.python.eager.profiler) is deprecated and will be removed after 2020-07-01.
Instructions for updating:
use `tf.profiler.experimental.stop` instead.
1270/1270 [==============================] - ETA: 0s - loss: 0.0019 - acc: 1.0000 - recall: 0.4882 - precision: 1.0000 - f1_score: 0.8549
Epoch 00001: val_f1_score improved from -inf to 0.83009, saving model to /app/_data/models/Eff02_2/eff02_2.h5
1270/1270 [==============================] - 607s 478ms/step - loss: 0.0019 - acc: 1.0000 - recall: 0.4882 - precision: 1.0000 - f1_score: 0.8549 - val_loss: 0.0029 - val_acc: 0.9635 - val_recall: 0.4837 - val_precision: 0.9910 - val_f1_score: 0.8301
Epoch 2/200
1270/1270 [==============================] - ETA: 0s - loss: 0

In [56]:
import kaggle

/bin/bash: import: command not found


In [57]:
! kaggle datasets init -p /app/_data/models/Eff02_2

Data package template written to: /app/_data/models/Eff01/dataset-metadata.json


In [60]:
! kaggle datasets create -p /app/_data/models/Eff01

Starting upload for file dict_birds.json
100%|██████████████████████████████████████| 5.86k/5.86k [00:01<00:00, 3.59kB/s]
Upload successful: dict_birds.json (6KB)
Starting upload for file eff0_1.h5
100%|███████████████████████████████████████| 52.7M/52.7M [01:21<00:00, 675kB/s]
Upload successful: eff0_1.h5 (53MB)
Skipping folder: .ipynb_checkpoints; use '--dir-mode' to upload folders
Your private Dataset is being created. Please check progress at https://www.kaggle.com/nataliayurasova/Eff01Initial
